<a href="https://colab.research.google.com/github/saisrikanthrayavarapu/PySpark/blob/master/Madrid_Air_Quality_index_uisng_HDFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.0-bin-hadoop3.tgz

!pip install -q findspark

!pip install -q pyspark

!pip install -q kaggle

     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 57.1 MB/s 


In [2]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
# Auto auth is not implemented, authorize google drive manually for execution

# mounting Google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir ~/.kaggle # create root folder for copying input file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d decide-soluciones/air-quality-madrid -f madrid.h5
# downloading the data set (CSV format) in the session

 81% 57.0M/70.3M [00:01<00:00, 53.3MB/s]
100% 70.3M/70.3M [00:01<00:00, 60.7MB/s]


In [7]:
# unzipping the file in the created directory
!unzip -q "/content/madrid.h5.zip"
# Removing the downloaded zip file from session storage
!rm /content/madrid.h5.zip

In [8]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [9]:
import pandas as pd
air_quality_df = pd.read_hdf('/content/madrid.h5', key='28079008') 
air_quality_df.head()

,BEN,CH4,CO,EBE,NMHC,NO,NO_2,NOx,O_3,PM10,PM25,SO_2,TCH,TOL
date,,,,,,,,,,,,,,
2001-07-01 01:00:00,30.65,NaN,6.91,42.639999,NaN,NaN,381.299988,1017.000000,9.010000,158.899994,NaN,47.509998,NaN,76.050003
2001-07-01 02:00:00,29.59,NaN,2.59,50.360001,NaN,NaN,209.500000,409.200012,23.820000,104.800003,NaN,20.950001,NaN,84.900002
2001-07-01 03:00:00,4.69,NaN,0.76,25.570000,NaN,NaN,116.400002,143.399994,31.059999,48.470001,NaN,11.270000,NaN,20.980000
2001-07-01 04:00:00,4.46,NaN,0.74,22.629999,NaN,NaN,116.199997,149.300003,23.780001,47.500000,NaN,10.100000,NaN,14.770000
2001-07-01 05:00:00,2.18,NaN,0.57,11.920000,NaN,NaN,100.900002,124.800003,29.530001,49.689999,NaN,7.680000,NaN,8.970000


In [10]:
air_quality_df.reset_index(inplace=True) 
air_quality_df['date'] = air_quality_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
air_quality_sdf = spark.createDataFrame(air_quality_df)

In [12]:
air_quality_sdf.select('date', 'NOx').show(5)

+-------------------+------------------+
|               date|               NOx|
+-------------------+------------------+
|2001-07-01 01:00:00|            1017.0|
|2001-07-01 02:00:00|409.20001220703125|
|2001-07-01 03:00:00|143.39999389648438|
|2001-07-01 04:00:00| 149.3000030517578|
|2001-07-01 05:00:00|124.80000305175781|
+-------------------+------------------+
only showing top 5 rows



In [13]:
air_quality_sdf.createOrReplaceTempView("air_quality_sdf")

In [14]:
sql_create_database = """
create database if not exists analytics
location '/user/cloudera/analytics/'
"""
result_create_db = spark.sql(sql_create_database)

In [15]:
sql_create_table = """ 
create table if not exists analytics.pandas_spark_hive 
using parquet as select 
to_timestamp(date) as date_parsed, * 
from air_quality_sdf
"""
result_create_table = spark.sql(sql_create_table)

In [16]:
spark.sql("select * from analytics.pandas_spark_hive").select("date_parsed", "O_3").show(5)

+-------------------+------------------+
|        date_parsed|               O_3|
+-------------------+------------------+
|2001-07-01 01:00:00| 9.010000228881836|
|2001-07-01 02:00:00| 23.81999969482422|
|2001-07-01 03:00:00|31.059999465942383|
|2001-07-01 04:00:00|23.780000686645508|
|2001-07-01 05:00:00|29.530000686645508|
+-------------------+------------------+
only showing top 5 rows

